## PROJECT-4. Решение комплексной бизнес-задачи 
##### Работу выполнила: Машковцева Дарья

##### ЦЕЛЬ
Подготовить основу рекомендательной системы.

##### ЗАДАЧИ
* Подготовить и проанализировать данные с помощью SQL.
* Обработать данные средствами Python.
* Составить итоговую таблицу с рекомендациями.
* Сделать вывод.

##### 1. SQL-запрос

*Подготовим данные по продажам курсов в разрезе пользователей, купивших более одного курса.*

*Выгрузим полученные данные в файл формата csv.*

In [26]:
'''
WITH user_count_cart AS
(SELECT ca.user_id, COUNT(DISTINCT it.resource_id) AS count_cart
FROM final.carts ca
    JOIN final.cart_items it ON ca.id=it.cart_id
WHERE ca.state = 'successful' AND it.resource_type = 'Course'
GROUP BY 1
HAVING COUNT(DISTINCT it.resource_id)>1),

user_resource AS
(SELECT ca.user_id, it.resource_id
FROM final.carts ca
    JOIN final.cart_items it ON ca.id=it.cart_id
WHERE ca.state = 'successful' AND it.resource_type = 'Course')

SELECT user_id, resource_id
FROM user_count_cart LEFT JOIN user_resource USING(user_id)
'''

"\nWITH user_count_cart AS\n(SELECT ca.user_id, COUNT(DISTINCT it.resource_id) AS count_cart\nFROM final.carts ca\n    JOIN final.cart_items it ON ca.id=it.cart_id\nWHERE ca.state = 'successful' AND it.resource_type = 'Course'\nGROUP BY 1\nHAVING COUNT(DISTINCT it.resource_id)>1),\n\nuser_resource AS\n(SELECT ca.user_id, it.resource_id\nFROM final.carts ca\n    JOIN final.cart_items it ON ca.id=it.cart_id\nWHERE ca.state = 'successful' AND it.resource_type = 'Course')\n\nSELECT user_id, resource_id\nFROM user_count_cart LEFT JOIN user_resource USING(user_id)\n"

##### 2. Обработка данных

In [27]:
import pandas as pd
import numpy as np

user_data = pd.read_csv('data/query_result_2022-02-04T17_24_08.397175Z.csv', sep=',')

*Найдем количество уникальных курсов из нашего датафрейма.*

In [28]:
unique_course = np.unique(user_data['resource_id']) 
len(unique_course)

126

*Сгруппируем данные по пользователям.*

In [29]:
group_data = user_data.groupby('user_id')['resource_id'].apply(lambda x: list(np.unique(x))).reset_index()
group_data.head()

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[356, 357, 1125]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"


*С помощью модуля itertools разобьём все покупки курсов по парам. Получим список кортежей.*

In [30]:
import itertools

list_pairs = list()
for res in group_data['resource_id']:
    for pair in itertools.combinations(res, 2):
        list_pairs.append(pair)

*Посчитаем для каждой пары количество ее вхождений в список.*

*Для этого будет использовтаь класс Counter из модуля collections. В результате получим словарь, где ключами будут являться пары курсов, а значениями - полученное количество вхождений, т.е. популярность пары курсов.* 

In [31]:
from collections import Counter

dist_pairs = Counter(list_pairs)

*Преобразуем словарь в датафрейм. Переименуем названия столбцов. Отсортируем по убыванию популярности пары курсов.*

In [32]:
pairs_data = pd.DataFrame.from_dict(dist_pairs, orient='index').reset_index()
pairs_data.columns = ['pair', 'count']
pairs_data = pairs_data.sort_values(by='count', ascending=False)
pairs_data.head()

,pair,count
186,"(551, 566)",797
381,"(515, 551)",417
205,"(489, 551)",311
105,"(523, 551)",304
519,"(566, 794)",290


*Для дальнейшей работы разобьем пару на отдельные столбцы. Лишний столбец удалим.*

In [33]:
pairs_data['first'] = pairs_data.apply(lambda x: x[0][0], axis=1)
pairs_data['second'] = pairs_data.apply(lambda x: x[0][1], axis=1)
pairs_data = pairs_data.drop(columns='pair', axis=1)
pairs_data.head()

,count,first,second
186,797,551,566
381,417,515,551
205,311,489,551
105,304,523,551
519,290,566,794


##### 3. Построение таблицы с рекомендациями

*Установим минимальную границу - какое количество раз считать слишком малым для определения рекомендации.*

*Вычислим верхнюю квартиль, которая отделяет 25% наблюдений с наибольшими значениями от остальных 75%.*

In [54]:
q_75 = np.percentile(pairs_data['count'], 75)

*Напишем функцию-поиск двух самых популярных курсов-рекомендаций для заданного курса.*

*На вход функции поступает курс, для которого осуществляется поиск, датафрейм и минимальная граница, для определения рекомендации.*

In [55]:
def search_popular_pair(course, df, min_popular):
    try:
        max_popular = 0 
        max_popular_2 = 0 
        recommendation = 0 
        recommendation_2 = 0 
        #проходим по всему датафрейму и определение рекомендации №1 и рекомендации №2 для исходного курса
        for i in df.index:
            if course == df['first'][i]:
                if df['count'][i] > max_popular:
                    max_popular_2 = max_popular
                    recommendation_2 = recommendation
                    max_popular = df['count'][i]
                    recommendation = df['second'][i]
                elif df['count'][i] > max_popular_2:
                    max_popular_2 = df['count'][i]
                    recommendation_2 = df['second'][i]
                    
            elif course == df['second'][i]:
                if df['count'][i] > max_popular:
                    max_popular_2 = max_popular
                    recommendation_2 = recommendation
                    max_popular = df['count'][i]
                    recommendation = df['first'][i]
                elif df['count'][i] > max_popular_2:
                    max_popular_2 = df['count'][i]
                    recommendation_2 = df['first'][i]
        
        # если обе рекомендации найдены и удовлетворяют минимальной границе, то возвращаем их            
        if max_popular > min_popular and max_popular_2 > min_popular:      
            return recommendation, recommendation_2
        # если рекомендация №2 не удовлятворяет минимальной границе, то определяем для нее новые рекомендации №2.1 и №2.2
        elif max_popular > min_popular and max_popular_2 <= min_popular:
            recom = search_popular_pair(recommendation_2, df, min_popular)
            #если рекомендация №2.1 не совпадает с рекомендацией №1, то возвращаем их. Иначе возвращаем рекомендацию №2.2
            if recommendation != recom[0]:
                return recommendation, recom[0]
            else: return recommendation, recom[1]
        # если рекомендация №1 не удовлетворяет минимальной границе, то определяем для нее новые рекомендации №1.1 и №1.2
        elif max_popular <= min_popular:
            recom = search_popular_pair(recommendation, df, min_popular)
            # если рекомендации №1.1 и №1.2 не совпадают с исходным курсом, то возвращаем их
            if course != recom[0] and course != recom[1]:
                return recom[0], recom[1]
            else: return 0

    except BaseException:
        print('ошибка')

*Найдем с помощью функции search_popular_pair рекомендации для каждого курса и оформим итоговую таблицу, состоящую из:*
* *курса, к которому идет рекомендация;*
* *курса для рекомендации №1 (самого популярного);*
* *курса для рекомендации №2 (второго по популярности).*

In [63]:
recom_course = pd.DataFrame(unique_course)
recom_course.columns = ['course']
recom_course['recommendation_1'] = 0
recom_course['recommendation_2'] = 0

for i in recom_course.index:
    course = recom_course['course'][i]
    recom = search_popular_pair(course, pairs_data, q_75)            
    recom_course['recommendation_1'][i] = recom[0]
    recom_course['recommendation_2'][i] = recom[1]

##### *Таблица с рекомендациями готова!*

In [64]:
recom_course

,course,recommendation_1,recommendation_2
0,356,571,357
1,357,571,356
2,358,570,752
3,359,570,358
4,360,745,516
...,...,...,...
121,1188,1141,513
122,1198,1125,357
123,1199,551,515
124,1200,566,551


*Сохраним итоговую таблицу в файл.*

In [58]:
recom_course.to_csv('data/recom_course.csv', index=False, sep=';')

##### 4. Выводы

*Была построена основа рекомендательной системы - таблица с двумя рекомендациями.*

*Благодаря данной таблицы можно предлагать клиентам интересные им курсы и повышать продажи.*

*По данным таблицы самыми часто рекомендованными курсами стали курсы с номерами 551, 566, 515. Можно выдвинуть предложение о дальнейшем увеличении стоимости данных курсов для большей прибыли компании. Но для проверки этого требуется дополнительный анализ.*